In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import Model

from keras.applications.resnet50 import ResNet50


Using TensorFlow backend.


In [2]:

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'category_dataset/'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 1
batch_size = 16


In [3]:
# Generate a model with all layers (with top)
vgg16 = applications.VGG16(weights='imagenet', include_top=False, input_shape = (img_width, img_height,3))

#Add a layer where input is the output of the  second last layer 
x = Flatten()(vgg16.layers[-1].output)
x = Dense(3, activation='softmax', name='predictions')(x)

#Then create the corresponding model 
model = Model(inputs=vgg16.input, outputs=x)
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:18]:
    layer.trainable = False


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [ ]:
import os
for root, subdirs, files in os.walk("./category_dataset/"):
    for f in files:
        r = f.replace(" ","")
        if( r != f):
            os.rename(root+"/"+f,root+"/"+r)
            print (f)


In [5]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)


Found 289 images belonging to 3 classes.


In [11]:
import os
error = True
while error == True:
    error = False
    train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
    
    try:
        i = 0
        for batch in iter(train_generator):
            print("HI")
            if i > len(train_generator.filenames)/batch_size + 2:
                break
            i+=1
    except Exception as e:
        filename = str(e).split("'")[1]
        print (f"removing {filename}")
        os.remove(filename)
        error = True

Found 289 images belonging to 3 classes.
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI


In [12]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    nb_val_samples=nb_validation_samples)

/home/amir/Projects/mercari/model_server/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/amir/Projects/mercari/model_server/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1, validation_data=<keras_pre..., steps_per_epoch=125, validation_steps=800)`
  import sys


Epoch 1/1
124/125 [============================>.] - ETA: 3s - loss: 0.7601 - acc: 0.6971

KeyboardInterrupt: 

In [13]:
model.save("category.model")

In [14]:
train_generator.num_classes

3

In [20]:
train_generator.class_indices

{'T-shirt': 0, 'hoodie': 1, 'pants': 2}

In [22]:
{v:k for k,v in train_generator.class_indices.items()}

{0: 'T-shirt', 1: 'hoodie', 2: 'pants'}

In [23]:
import pickle

In [ ]:
pickle.dump()

In [33]:
import os
for root, subdirs, files in os.walk("./category_dataset/"):
    for f in files:
        print (f)
        r = f.replace(" ","")
        if( r != f):
            os.rename(root+"/"+f,root+"/"+r)

15. 15105_federatemodcargo_c_510.jpg
28. m-original-mountain-pant-relaxed-fit-organic-ranch.jpg
52. pol_pl_spodnie-minimalist-gore-tex-pants-women-31145_2.jpg
76. 14700_flightpants_v2_a_510.jpg
69. spodnie-black-diamond-helio-active-pants-midnight.jpg
97. tiro_17_training_pants_black_bs3693_25_model.jpg
12. 52370l_001_f?$dw-product$=.jpg
34. m304ghow_rge_1.jpg
42. attackpant_bl_fr_1425x1825.jpg
73. big_spodnie-w-s-patagonia-crestview-pants-black-2.jpg
35. 95016_stn.jpg
70. spodnie_tiro17_training_pants_czern_bs3693_22_model.jpg
17. 196260081-front-pdp.jpg
33. bmc-terra-pants-limited-edition-p696-15369_image.jpg
92. m6a262f4a0df6b95a7155eea2f0f38fb4_409991_z_1.jpg
79. jay_tunic_valerie_pants_-1-.jpg
57. guest_ec69e626-e349-443f-a0cf-49514b162694?wid=488&hei=488&fmt=pjpeg.jpg
4. e6687_br5646.jpg
13. 71703_brn.jpg
66. selby_pants_amarantowe_1_169_pln_4.jpg
85. fokhb3hhzp5qvmpjtp0ezc3dihunrns8-34.jpg
20. 20tz_16_1905_cornstalk_main.jpg
27. goods_03_407868?$prod$.jpg
26. 15771_bah.jpg
41. d